In [25]:
from sodapy import Socrata
import pandas as pd
import mysql.connector as mysql
import json
import time 

In [43]:
#function to connect ot mysql 
def connect_db():
    try : 
        data = open("DB_config.txt","r").read()
        data = json.loads(data)
    except : 
        print("could not read DB_config,txt file")
        
    try:
        db = mysql.connect(
            host = data["host"],
            user = data["user"],
            password = data["password"],
            database = data["database"]
        )
        
        return db
    except:
        print("could not access the database")

In [3]:
def create_tables(cursor):
    cursor.execute("SHOW TABLES")
    current_tables = list(cursor)
    current_tables = list(map(lambda x:x[0] , current_tables))
    if "pd_cnts_snsrs" not in current_tables:
        Q1 = '''CREATE TABLE pd_cnts_snsrs(sensor_id INT PRIMARY KEY, 
            place VARCHAR(100), 
            lattitude FLOAT, 
            longitude FLOAT)
            '''
        
        cursor.execute(Q1)
        print("pd_cnts_snsrs table created")
    else:
        print("pd_cnts_snsrs table already exists")
        
    if "pd_cnts" not in current_tables:
        Q2 = '''CREATE TABLE pd_cnts(
        sensor_id INT , 
        pedestrian_count INT , 
        date_time DATETIME ,
        PRIMARY KEY(sensor_id , date_time))'''
        
        Q3 = "ALTER TABLE pd_cnts ADD FOREIGN KEY(sensor_id) REFERENCES pd_cnts_snsrs(sensor_id)"
        cursor.execute(Q2)
        cursor.execute(Q3)
        print("pd_cnts table created")
    else : 
        print("pd_cnts already exists")

In [4]:
def populate_sensors(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC" ):
    client = Socrata("data.melbourne.vic.gov.au", access_token)
    results = client.get("h57g-5234")
    results_df = pd.DataFrame.from_records(results)
    sensors = results_df[["sensor_id" , "sensor_description" , "latitude" , "longitude"]]
    # sensors.iloc[:,0] = sensors.sensor_id.astype("int")
    # sensors.iloc[:,1] = sensors.sensor_description.astype("str")
    # sensors.iloc[:,2] = sensors.latitude.astype("float")
    # sensors.iloc[:,3] = sensors.longitude.astype("float")
    sensors_list = [tuple(x) for x in sensors.to_numpy()]
    
#     if sensors_list:
#         Q4a = "DELETE from pd_cnts_snsrs"
#         cursor.execute(Q4a)

    Q4 = "INSERT INTO pd_cnts_snsrs(sensor_id , place, lattitude , longitude) VALUES (%s , %s , %s , %s)"
    cursor.executemany(Q4 , sensors_list)
    print("successfully added data to pd_cnts_snsrs table ")
    return 1

In [58]:
def populate_sensors(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC" ):
    client = Socrata("data.melbourne.vic.gov.au", access_token)
    results = client.get("h57g-5234")
    results_df = pd.DataFrame.from_records(results)
    sensors = results_df[["sensor_id" , "sensor_description" , "latitude" , "longitude"]]
    sensors_list = [tuple(x) for x in sensors.to_numpy()]

    if sensors_list:
        Q5a = "DROP TABLE pd_cnts"
        cursor.execute(Q5a)
        
        cursor.execute("TRUNCATE pd_cnts_snsrs")
        
        Q2 = '''CREATE TABLE pd_cnts(
        sensor_id INT , 
        pedestrian_count INT , 
        date_time DATETIME ,
        PRIMARY KEY(sensor_id , date_time))'''
        
        Q3 = "ALTER TABLE pd_cnts ADD FOREIGN KEY(sensor_id) REFERENCES pd_cnts_snsrs(sensor_id)"
        cursor.execute(Q2)
        cursor.execute(Q3)

        Q4 = "INSERT INTO pd_cnts_snsrs(sensor_id , place, lattitude , longitude) VALUES (%s , %s , %s , %s)"
        cursor.executemany(Q4 , sensors_list)
        print("successfully added data to pd_cnts_snsrs table ")
        return 1
    else:
        print("No new data to add")

In [59]:
def populate_sensors_data(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC"):
    
    client = Socrata("data.melbourne.vic.gov.au",access_token)
    results = client.get("d6mv-s43h")
    results_df = pd.DataFrame.from_records(results)
    sensor_data = results_df[["sensor_id"  , "total_of_directions" , "date_time"]]
    sensors_data_list = [tuple(x) for x in sensor_data.to_numpy()]
    
    if sensors_data_list:
#         print("dropping pd_cnts")
#         Q5a = "DROP TABLE pd_cnts"
#         cursor.execute(Q5a)
        
#         Q2 = '''CREATE TABLE pd_cnts(
#         sensor_id INT , 
#         pedestrian_count INT , 
#         date_time DATETIME ,
#         PRIMARY KEY(sensor_id , date_time))'''
        
#         Q3 = "ALTER TABLE pd_cnts ADD FOREIGN KEY(sensor_id) REFERENCES pd_cnts_snsrs(sensor_id)"
#         cursor.execute(Q2)
#         cursor.execute(Q3)
        cursor.execute("TRUNCATE pd_cnts")
    
        Q5 = "INSERT INTO pd_cnts(sensor_id , pedestrian_count , date_time) VALUES (%s, %s , %s)"
        cursor.executemany(Q5 , sensors_data_list)
    
        print("successfully added new data to pd_cnts table")
        return 1

    print("no new data recieved")

In [60]:
db = connect_db()
cursor = db.cursor()
create_tables(cursor)
db.commit()
time.sleep(3)
populate_sensors(cursor)
db.commit()
time.sleep(3)
populate_sensors_data(cursor)
db.close()

pd_cnts_snsrs table already exists
pd_cnts already exists
successfully added data to pd_cnts_snsrs table 
successfully added new data to pd_cnts table


In [14]:
cursor.execute("SELECT * FROM pd_cnts")
list(cursor)

In [23]:
db.close()

In [44]:
db = connect_db()

In [47]:
cursor = db.cursor()
cursor.execute("SELECT * from pd_cnts")
list(cursor)

[(1, 5, datetime.datetime(2021, 3, 31, 19, 26)),
 (1, 9, datetime.datetime(2021, 3, 31, 19, 27)),
 (1, 6, datetime.datetime(2021, 3, 31, 20, 12)),
 (1, 5, datetime.datetime(2021, 3, 31, 20, 14)),
 (1, 4, datetime.datetime(2021, 3, 31, 20, 15)),
 (1, 7, datetime.datetime(2021, 3, 31, 20, 16)),
 (1, 8, datetime.datetime(2021, 3, 31, 20, 17)),
 (1, 7, datetime.datetime(2021, 3, 31, 20, 18)),
 (1, 11, datetime.datetime(2021, 3, 31, 20, 19)),
 (1, 11, datetime.datetime(2021, 3, 31, 20, 20)),
 (1, 6, datetime.datetime(2021, 3, 31, 20, 21)),
 (1, 6, datetime.datetime(2021, 3, 31, 20, 22)),
 (1, 9, datetime.datetime(2021, 3, 31, 20, 23)),
 (1, 6, datetime.datetime(2021, 3, 31, 20, 24)),
 (1, 4, datetime.datetime(2021, 3, 31, 20, 25)),
 (1, 5, datetime.datetime(2021, 3, 31, 20, 26)),
 (2, 10, datetime.datetime(2021, 3, 31, 19, 26)),
 (2, 4, datetime.datetime(2021, 3, 31, 19, 32)),
 (2, 1, datetime.datetime(2021, 3, 31, 19, 35)),
 (2, 4, datetime.datetime(2021, 3, 31, 19, 38)),
 (2, 6, datetime.